## Analyze whether SNWD varies more from year to year or from place to place.

In [2]:
import pandas as pd
import numpy as np
import urllib
import math

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [27]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBSSBBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
502
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|-8864.636346120042|2039.9548861316832| 698.3849072461767|   1797.4|SBSSBBBS|  43.465|-113.5581|       SNWD|0.08489938379981538|0.03534173056103782|0.03086802364758513|0.748963242539

In [28]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [29]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00102260,1993.0,-8864.636346
1,USC00104845,1997.0,-6921.010051
2,USC00104845,1980.0,-5862.926829
3,USC00104230,1972.0,-6048.058037
4,USC00103942,1952.0,-9637.720618


In [30]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,USC00100010,USC00100227,USC00100375,USC00100915,USC00102260,USC00102676,USC00102823,USC00103110,USC00103297,USC00103825,...,USC00108380,USC00108535,USC00108906,USC00108937,USC00109601,USC00480124,USW00024142,USW00024145,USW00024156,USW00094143
year,,,,,,,,,,,,,,,,,,,,,
1911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,-575.6836,-575.867863,NaN,-605.601275,NaN,NaN,452.695503,NaN,980.993082,NaN,...,-2400.071258,NaN,-5238.143526,NaN,NaN,NaN,-2824.029765,-1159.463215,190.590171,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1737.104031,...,319.980859,NaN,-5379.110928,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4703.194395,...,621.838765,NaN,-5574.969694,NaN,NaN,NaN,894.272397,NaN,NaN,NaN
1952.0,NaN,NaN,NaN,820.536368,NaN,NaN,NaN,NaN,NaN,-7141.120392,...,-1711.255436,NaN,-10869.006570,NaN,NaN,NaN,-1191.162994,-428.336176,830.514876,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3256.566677,...,NaN,NaN,-4428.479870,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3234.758510,...,NaN,NaN,-4295.499467,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,833.893021,NaN,-1784.402002,NaN,NaN,NaN,NaN,-389.165102,NaN,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-3803.643704,NaN,NaN,NaN,NaN,833.875514,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [31]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  2765.11652981
RMS removing mean-by-station=  1528.810233
RMS removing mean-by-year   =  2144.76650138


In [16]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 2765.11652981
0 after removing mean by year    = 2144.76650138
0 after removing mean by stations= 1326.37161724
1 after removing mean by year    = 1143.59923609
1 after removing mean by stations= 1084.93331804
2 after removing mean by year    = 1063.94560272
2 after removing mean by stations= 1056.03526849
3 after removing mean by year    = 1052.95666731
3 after removing mean by stations= 1051.73034413
4 after removing mean by year    = 1051.23234929
4 after removing mean by stations= 1051.0264882
